In [16]:
import spacy
# python -m spacy download en_core_web_sm
# python -m spacy download pl_core_news_sm
import re
import tarfile
import os
import matplotlib.pyplot as plt
import numpy as np
import regex
import pandas as pd
import morfeusz2
from elasticsearch_dsl import *
from elasticsearch_dsl import query
from elasticsearch import *
import os
import pickle

import spacy
from collections import Counter
from spacy.tokenizer import *

1. Make sure the texts in the corpus does not contain HTML code.
2. Use SpaCy [tokenizer API](https://spacy.io/api/tokenizer) to tokenize the text from the cleaned law corpus.
3. Compute **frequency list** for each of the processed files.
4. Aggregate the result to obtain one global frequency list.
5. Reject all entries that are shorter than 2 characters or contain non-letter characters (make sure to include Polish
   diacritics).
6. Make a plot in a logarithmic scale (for X and Y):
   1. X-axis should contain the **rank** of a term, meaning the first rank belongs to the term with the highest number of
      occurrences; the terms with the same number of occurrences should be ordered by their name,
   2. Y-axis should contain the **number of occurrences** of the term with given rank.
7. Install [Morfeusz](http://morfeusz.sgjp.pl/download/) (Binding dla Pythona) and use it to find all words that do not
   appear in that dictionary.
8. Find 30 words with the highest ranks that do not belong to the dictionary.
9. Find 30 random words (i.e. shuffle the words) with 5 occurrences that do not belong to the dictionary.
10. Use Levenshtein distance and the frequency list, to determine the most probable correction of the words from
   lists defined in points 8 and 9. (**Note**: You don't have to apply the distance directly. Any method that is more efficient than scanning the
   dictionary will be appreciated.)
11. Load [SGJP dictionary](http://morfeusz.sgjp.pl/download/) (Słownik SGJP dane tekstowe) to ElasticSearch (one document for each form) 
   and use fuzzy matching to obtain the possible
   corrections of the 30 words with 5 occurrences that do not belong to the dictionary.
12. Compare the results of your algorithm and output of ES. 
13. Draw conclusions regarding:
   * the distribution of words in the corpus,
   * the number of true misspellings vs. the number of unknown words,
   * the performance of your method compared to ElasticSearch,
   * the results provided by your method compared to ElasticSearch,
   * the validity of the obtained corrections.

In [17]:
nlp = spacy.load("pl_core_news_sm")
tokenizer = Tokenizer(nlp.vocab)


In [86]:

file_tokens = {}

i = 0
path="../data/ustawy"
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r',encoding='utf-8') as f: # open in readonly mode
        text = f.read()
        text = regex.sub(r'\s+', ' ',text)

        doc = tokenizer(text)
        words = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
#         words = [x for x in words if len(x)>1 and ]
        file_tokens[f.name] = words
        word_freq = Counter(words)
        i+=1
        if i % 200 == 0:
            print(i)
    

200
400
600
800
1000


In [87]:
word_freq ={}
for filename, tokens in file_tokens.items():
    word_freq[filename] = Counter(tokens)

In [61]:
word_freq_total = Counter({})
for counting in word_freq.values():
    word_freq_total.update(counting)


In [88]:
word_freq_total.most_common(5)

[('\n', 181703),
 ('\n  ', 54885),
 ('\n     ', 54047),
 ('ust.', 53074),
 ('art.', 52161)]

In [84]:
import regex
' Hello \n\n Workd '.strip()
text = regex.sub(r'\s+', ' ',text)
print(text)

 Dz.U. z 1998 r. Nr 117, poz. 759 USTAWA z dnia 25 lipca 1998 r. o zmianie ustawy o systemie oświaty Art. 1. W ustawie z dnia 7 września 1991 r. o systemie oświaty (Dz.U. z 1996 r. Nr 67, poz. 329, Nr 106, poz. 496 oraz z 1997 r. Nr 28, poz. 153 i Nr 141, poz. 943) wprowadza się następujące zmiany: 1) w art. 1: a) po pkt 5 dodaje się pkt 5a w brzmieniu: "5a) opiekę nad uczniami ze znacznymi lub sprzężonymi dysfunkcjami poprzez umożliwianie realizowania indywidualnych form i programów nauczania,", b) w pkt 7 skreśla się wyraz "średnich"; 2) w art. 2: a) pkt 1 otrzymuje brzmienie: "1) przedszkola, w tym z oddziałami integracyjnymi oraz przedszkola specjalne,", b) pkt 2 otrzymuje brzmienie: "2) szkoły podstawowe, gimnazja i szkoły ponadgimnazjalne ogólnokształcące i zawodowe, w tym z oddziałami integracyjnymi, z wyjątkiem szkół wyższych, oraz szkoły specjalne, szkoły sportowe oraz mistrzostwa sportowego i szkoły artystyczne,", c) pkt 3 otrzymuje brzmienie: "3) placówki oświatowo-wychowawc

In [75]:
import string
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r',encoding='utf-8') as f: # open in readonly mode
        text = f.read()
#         text =text.translate(str.maketrans('', '', string.punctuation+'\n'))

        doc = tokenizer(text)
        words = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
        print(words)
        break


['\n\n\n\n', 'Dz.U.', '1998', 'r.', 'Nr', '117,', 'poz.', '759', '                                     \n                                        \n                                        \n                                        \n                                        \n                                        \n                                     ', 'USTAWA', '\n                             ', 'dnia', '25', 'lipca', '1998', 'r.', '\n                                        \n                       ', 'zmianie', 'ustawy', 'systemie', 'oświaty', '\n                                        \n                                        \n                                     ', 'Art.', '1.', '\n', 'ustawie', 'dnia', '7', 'września', '1991', 'r.', 'systemie', 'oświaty', '(Dz.U.', '1996', 'r.', 'Nr', '67,', '\n', 'poz.', '329,', 'Nr', '106,', 'poz.', '496', '1997', 'r.', 'Nr', '28,', 'poz.', '153', 'Nr', '141,', 'poz.', '943)', '\n', 'wprowadza', 'następujące', 'zmiany:', '\n', '1)', 'art.', '1:

In [56]:
word_freq_all.most_common(4)

AttributeError: 'dict' object has no attribute 'most_common'

In [47]:
#multiprocessing python 
list(word_freq.values())[0].most_common(5)


[('\n       ', 123),
 ('ust.', 116),
 ('\n           ', 99),
 ('art.', 98),
 ('\n', 83)]

In [58]:
list(word_freq.values()).

Counter({'\n\n\n\n': 1,
         'Dz.U.': 1,
         '1998': 2,
         'r.': 11,
         'Nr': 6,
         '117,': 1,
         'poz.': 6,
         '759': 1,
         '                                     \n                                        \n                                        \n                                        \n                                        \n                                        \n                                     ': 1,
         'USTAWA': 1,
         '\n                             ': 1,
         'dnia': 6,
         '25': 1,
         'lipca': 1,
         '\n                                        \n                       ': 1,
         'zmianie': 1,
         'ustawy': 6,
         'systemie': 2,
         'oświaty': 35,
         '\n                                        \n                                        \n                                     ': 1,
         'Art.': 10,
         '1.': 17,
         '\n': 83,
         'ustawie': 4,
         '7'

In [42]:
print(type(list(file_tokens.values()[0]))

SyntaxError: unexpected EOF while parsing (<ipython-input-42-b7c2ede85770>, line 1)

In [45]:
Counter(['ala','ma', 'ma' ,'kota'])

Counter({'ala': 1, 'ma': 2, 'kota': 1})

In [ ]:
with open('file_tokens.pkl', 'wb') as outp:
    pickle.dump(file_tokens, outp, pickle.HIGHEST_PROTOCOL)


In [3]:
nlp = spacy.load("en_core_web_sm")
tokens = nlp("dog cat banana afskfsd")

for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

dog True 4.8634477 True
cat True 5.2525845 True
banana True 5.3935223 True
afskfsd True 5.5341754 True


In [17]:
from spacy.lang.pl.examples import sentences 

nlp = spacy.load("pl_core_news_sm")
doc = nlp(sentences[0])
print(dir(doc))
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

['_', '__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '_bulk_merge', '_get_array_attrs', '_py_tokens', '_realloc', '_vector', '_vector_norm', 'cats', 'char_span', 'copy', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_dict', 'from_disk', 'from_docs', 'get_extension', 'get_lca_matrix', 'has_annotation', 'has_extension', 'has_unknown_spaces', 'has_vector', 'is_nered', 'is_parsed', 'is_sentenced', 'is_tagged', 'lang', 'lang_', 'mem', 'noun_chunks', 'noun_chunks_iterator', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_ents', 'set_extension', 'similarity', 'spans', 'tensor', 'text', 'text_with_ws'

In [12]:
iterations = 0
path="../data/ustawy"
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r',encoding='utf-8') as f: # open in readonly mode
        text = f.read()
        doc = nlp(text)
        iterations+=1
        if iterations==10:
            break
    



In [13]:
import spacy
nlp = spacy.load("en_core_web_sm")

list_of_words = ['Pies', 'jump', 'catch']

def word_count(string):
    words_counted = 0
    my_string = nlp(string)

    for token in my_string:
        # actual word
        word = token.text
        # lemma
        lemma_word = token.lemma_
        # part of speech
        word_pos = token.pos_
        if lemma_word in list_of_words:
            words_counted += 1
            print(lemma_word)
    return words_counted


sentence = "I ran, jumped, and caught the ball."
words_counted = word_count(sentence)
print(words_counted)

jump
catch
2


In [10]:

nlp = spacy.load("pl_core_news_sm")
# text = 
doc = nlp(text)
#remove stopwords and punctuations
words = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
word_freq = Counter(words)
common_words = word_freq
print (common_words)


NameError: name 'text' is not defined

In [5]:
from spacy import *
from collections import Counter
nlp = spacy.load("pl_core_news_sm")

texts = ["One document.", "...", "Lots of documents"]
for doc in tokenizer.pipe(texts, batch_size=50):
    print(doc)
    # pass


AttributeError: module 'spacy.tokenizer' has no attribute 'pipe'

In [39]:

import spacy
from spacy.symbols import ORTH
nlp = spacy.load("pl_core_news_sm")
doc = nlp("lemme that")
print([w.text for w in doc])

special_case = [{ORTH: "lem"}, {"ORTH": "me"}]
nlp.tokenizer.add_special.case("lemme", special_case)
print([w.text for w in nlp("lemme that")])

['lemme', 'that']


AttributeError: 'spacy.tokenizer.Tokenizer' object has no attribute 'add_special'